In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pdb

In [ ]:
# INPUT PARAMETERS

def input_param():
    w = 1000000         # time window [in ms]
    g = 0.1             # gamma parameter
    d = 10              # base difficulty
    MAX_TX = 5000       # number of txs generated
    
    return w, g, d, MAX_TX

In [ ]:
####### SINGLE NODE ANALYSIS WITH ADAPTIVE POW ########
#######################################################

tx = []
delay = []
dif = []
mu = 10**-7      # CPU power

[w, g, d, MAX_TX] = input_param()

r = 0.3 + 0.4*np.random.rand(MAX_TX)
x = d

for i in range(MAX_TX):

    # PoW time (40 is a scale factor to convert time in ms)
    delay.append(mu*3**x*r[i]/40)
    if i > 0:
        tx.append(tx[i-1] + mu*3**x*r[i]/40)
    else:
        tx.append(mu*3**x*r[i]/40)
    dif.append(x)
    
    # adapt difficulty
    a = (sum(1 for j in tx if j <= tx[i] and j > max(tx[i] - w, 0)))    # compute how many txs in the prev time window
    x = d + np.floor(a*g)                                               # set the new difficulty
    
    if i % 500 == 0:
        print(i)

In [ ]:
# PoW difficulty plot

fig = plt.figure()
plt.plot(dif)
plt.xlabel("Sequence of txs")
plt.ylabel("PoW difficulty")
plt.grid()
plt.show()

In [ ]:
# Time to compute the PoW

fig = plt.figure()
plt.plot(np.array(delay)/1000, linewidth=0.2)
plt.plot(np.mean(delay)*np.ones((MAX_TX))/1000, color="r", linewidth=2)
plt.xlabel("Sequence of txs")
plt.ylabel("PoW time")
plt.grid()
plt.show()

In [ ]:
# Throughput [tps]

fig = plt.figure()
plt.semilogy((np.array(range(MAX_TX))+1)*1000/np.array(tx))
plt.grid()
plt.xlabel("Sequence of txs")
plt.ylabel("Throughput")
plt.show()

In [ ]:
######## SINGLE NODE ANALYSIS WITH FIXED POW ##########
#######################################################

MAX_TX = 5000
tx = []
delay = []
mu = 10**-7     # CPU power

# set the PoW difficulty x as the base difficulty
x = 14

r = 0.3 + 0.4*np.random.rand(MAX_TX)

for i in range(MAX_TX):

    # PoW time (40 is a scale factor to convert time in ms)
    delay.append(mu*3**x*r[i]/40)
    if i > 0:
        tx.append(tx[i-1] + mu*3**x*r[i]/40)
    else:
        tx.append(mu*3**x*r[i]/40)
    
    if i % 500 == 0:
        print(i)

In [ ]:
# Time to compute the PoW

fig = plt.figure()
plt.plot(np.array(delay)/1000, linewidth=0.2)
plt.plot(np.mean(delay)*np.ones((MAX_TX))/1000, color="r", linewidth=2)
plt.xlabel("Sequence of txs")
plt.ylabel("PoW time")
plt.grid()
plt.show()

In [ ]:
# Throughput [tps]

fig = plt.figure()
plt.plot((np.array(range(MAX_TX))+1)*1000/np.array(tx))
plt.grid()
plt.xlabel("Sequence of txs")
plt.ylabel("Throughput")
plt.show()

In [ ]:
######## SENSITIVITY ANALYSIS ON GAMMA PARAMETER #########
##########################################################

MAX_TX = 2000
tps = []
mu_vector = [10**0, 10**-1, 10**-7]      # CPU power
g_vector = [0, 0.001, 0.01, 0.1, 0.2, 0.5, 1]
[w, _, d, _] = input_param()

for g in g_vector:

    print("Gamma = " + str(g))
    
    for mu in mu_vector:
    
        # set the PoW difficulty x as the base difficulty
        x = d
        tx = []
        r = 0.3 + 0.4*np.random.rand(MAX_TX)

        for i in range(MAX_TX):

            # PoW time (40 is a scale factor to convert time in ms)
            #delay.append(mu*3**x*r[i]/40)
            if i > 0:
                tx.append(tx[i-1] + mu*3**x*r[i]/40)
            else:
                tx.append(mu*3**x*r[i]/40)
            #dif.append(x)

            # adapt difficulty
            a = (sum(1 for j in tx if j <= tx[i] and j > max(tx[i] - w, 0)))    # compute how many txs in the prev time window
            x = d + np.floor(a*g)                                               # set the new difficulty

        print((MAX_TX+1)*1000/np.array(tx[MAX_TX - 1]))
        tps.append((MAX_TX+1)*1000/np.array(tx[MAX_TX - 1]))

In [ ]:
# Plot - sensitivity analysis of gamma parameter

t = np.reshape(tps, (7, 3))

fig = plt.figure(figsize=(10, 6))

plt.loglog(g_vector, t)
plt.grid()
plt.ylabel("throughput [tps]")
plt.xlabel("gamma")
plt.legend(["IoT", "Laptop", "FPGA"])
plt.show()